In [1]:
import pandas as pd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
load_dotenv()
import os
from datetime import datetime, timedelta
from tqdm import tqdm
from processor.processor import Processor as p
from database.adatabase import ADatabase

In [2]:
market = ADatabase("market")
market.connect()
index = market.retrieve("sp500")
market.disconnect()

In [3]:
start = datetime(2024,1,1)
end = datetime.now()

In [4]:
prices = []
date = start
market.connect()
for ticker in tqdm(index["ticker"]):
    try:
        price = market.query("prices",{"ticker":ticker})
        price = p.column_date_processing(price)
        price.sort_values("date",inplace=True)
        prices.append(price)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|████████████████████████                                                                                                                                                                     | 64/502 [00:02<00:18, 23.73it/s]

'date'


 15%|████████████████████████████▉                                                                                                                                                                | 77/502 [00:03<00:18, 23.19it/s]

'date'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 502/502 [00:23<00:00, 21.48it/s]

'date'


In [5]:
market_prices = pd.concat(prices)

In [12]:
def calculate_indicators(price, timeframe):
    """Calculate indicators for a single ticker."""
    price = price.sort_values("date")
    price["sma"] = price["adjclose"].rolling(timeframe).mean() / price["adjclose"]
    price["ema"] = price["adjclose"].ewm(span=timeframe, adjust=False).mean() / price["adjclose"]

    delta = price["adjclose"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(timeframe).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(timeframe).mean()
    rs = gain / loss
    price["rsi"] = 100 - (100 / (1 + rs))

    price["std"] = price["adjclose"].rolling(timeframe).std()
    price["bollinger_upper"] = (price["sma"] + 2 * price["std"]) / price["sma"]
    price["bollinger_lower"] = (price["sma"] - 2 * price["std"]) / price["sma"]

    price["pct_change"] = price["adjclose"].pct_change(periods=timeframe)
    price["coev"] = price["adjclose"].rolling(timeframe).std() / price["adjclose"].rolling(timeframe).mean()

    return price.dropna()

In [13]:
prices = []
timeframe = 100
for ticker in tqdm(market_prices["ticker"].unique()):
    try:
        price = market_prices[market_prices["ticker"] == ticker]
        price = calculate_indicators(price, timeframe)
        prices.append(price)
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Combine and sort all price data
sim = pd.concat(prices).dropna().sort_values("date")
sim["year"] = sim["date"].dt.year
sim["week"] = [x.week for x in sim["date"]]
sim["sell_price"] = sim["adjclose"]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 499/499 [01:24<00:00,  5.90it/s]


In [14]:
signals = ["sma"
           ,"ema","rsi","bollinger_upper","bollinger_lower","momentum","pct_change","coev"
          ]
ascendings = [True,False]

In [15]:
analysis = []
for signal in signals:
    for ascending in ascendings:
        opportunities = sim[sim["year"]>2012].groupby(["year","week","ticker"]).agg({"date":"last","adjclose":"first","sell_price":"last",signal:"first"}).reset_index()
        trades = opportunities.sort_values(signal,ascending=ascending).groupby(["year","week"]).first().reset_index()
        trades.sort_values("date",inplace=True)
        trades["return"] = trades["sell_price"] / trades["adjclose"]
        trades["pnl"] = trades["return"].cumprod()
        trades["downside"] = trades["pnl"].rolling(100).max() - trades["pnl"]
        analysis.append({
            "signal":signal,
            "ascending":ascending,
            "pnl":trades["pnl"].iloc[-1],
            "downside":trades["downside"].max(),
            "volatility": trades["return"].std()
        })

In [16]:
a = pd.DataFrame(analysis)
a["risk"] = a["downside"] * a["volatility"]

In [17]:
a.sort_values("risk",ascending=True).head(50)

,signal,ascending,pnl,downside,volatility,risk
7,bollinger_upper,False,7.329552,1.272168,0.044279,0.056330
8,bollinger_lower,True,7.329552,1.272168,0.044279,0.056330
14,coev,True,5.999878,2.730743,0.031424,0.085812
2,ema,True,16.765844,3.822623,0.080322,0.307043
5,rsi,False,2.496670,8.791324,0.066969,0.588749
0,sma,True,6.368253,7.900102,0.085195,0.673050
4,rsi,True,8.237979,10.179159,0.075466,0.768177
6,bollinger_upper,True,62.788638,13.728866,0.063741,0.875092
9,bollinger_lower,False,62.788638,13.728866,0.063741,0.875092
11,momentum,False,101.074272,33.143529,0.092066,3.051395
